# Example of Weak Label Learning with warm up

In [27]:
from datasets import openml_datasets,torch_datasets
from utils import weakener
from models import general_model

IndentationError: expected an indented block (general_model.py, line 28)

In [13]:
DS = openml_datasets.OpenML_Dataset('iris')

In [14]:
print(DS.num_features)
print(DS.num_classes)

4
tensor(3, dtype=torch.int32)


In [16]:
Weak = weakener.Weakener(DS.num_classes)

In [19]:
print(dir(Weak))

['M', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'c', 'generate_M', 'generate_weak', 'generate_wl_priors', 'label_matrix', 'pll_weights', 'virtual_labels', 'virtual_matrix', 'w', 'z']


In [21]:
print(DS.__dict__)

{'dataset': 'iris', 'tr_size': 0.7, 'weak_labels': None, 'batch_size': 64, 'shuffle': True, 'splitting_seed': 47, 'num_classes': tensor(3, dtype=torch.int32), 'train_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000027802F33FA0>, 'test_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000027802F333D0>, 'train_num_samples': 105, 'test_num_samples': 45, 'num_features': 4}


In [24]:
Weak.generate_M(model_class='pll')

In [26]:
print(DS.__dict__)

{'dataset': 'iris', 'tr_size': 0.7, 'weak_labels': None, 'batch_size': 64, 'shuffle': True, 'splitting_seed': 47, 'num_classes': tensor(3, dtype=torch.int32), 'train_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000027802F33FA0>, 'test_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000027802F333D0>, 'train_num_samples': 105, 'test_num_samples': 45, 'num_features': 4}
